<a href="https://colab.research.google.com/github/anilsolanki2645/WebScarping/blob/main/IMDB_Movie_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scraping Multiples pages of IMDB at a time to fetch top 1000 movies data.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* importing all the needed libraries

In [11]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import openpyxl
import matplotlib.pyplot as plt

* Declaring the variable of Headers

In [12]:
#Declaring the headers
headers = {"Accept-Language": "en-US,en;q=0.5"}
#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
time=[]
rating=[]
metascore =[]
votes = []
gross = []
description = []

In [13]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,1000,100)

In [ ]:
"""excel = openpyxl.Workbook()
print(excel.sheetnames)
sheet = excel.active
sheet.title = 'Top Rated Movies'
print(excel.sheetnames)"""

In [ ]:
"""try:
    #sheet.append([movie_name, year, time, rating, metascore, votes, gross, description])
    sheet.append(movie_name)
    print("success")
except ValueError:
    print("Handling the error case")"""


In [14]:
#print(movie_name)

### collect and parse the information using
* requests lib and BeautifulSoup lib
* Store the information in variable
* and append in a list

In [15]:
#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)

        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(year_of_release)

        runtime = store.p.find("span", class_ = 'runtime').text
        time.append(runtime)

        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)

        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else "*****"
        metascore.append(meta)


        value = store.find_all('span', attrs = {'name': "nv"})

        vote = value[0].text
        votes.append(vote)

        grosses = value[1].text if len(value)>1 else '%^%^%^'
        gross.append(grosses)

        # Description of the Movies -- Not explained in the Video, But you will figure it out.
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
        description.append(description_)



* creating a DataFrame

In [ ]:
#creating a dataframe
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time": time,"Movie Rating": rating, "Meatscore of movie": metascore, "Votes" : votes, "Gross": gross, "Description": description  })
movie_list.head(11)

In [ ]:
movie_list[-3:]

In [ ]:
type(movie_list)

* convert dataframe to Excel
* Create a Excel file

In [10]:
movie_list.to_excel('IMDB_Movie_Ratings.xlsx', header=True, index=False)